In [8]:
import pandas as pd
import numpy as np
import surprise
import os

ratings = pd.read_csv("C:/Users/User/Desktop/cdac advance modules/pml lecture/class  practice/program_dataset/u.data",sep='\t',
                      names = ['uid','iid','rating'])

In [9]:
ratings.head()

,uid,iid,rating
196,242,3,881250949
186,302,3,891717742
22,377,1,878887116
244,51,2,880606923
166,346,1,886397596


In [3]:

lowest_rating = ratings['rating'].min()
highest_rating = ratings['rating'].max()

print("Ratings range between {0} and {1}".format(lowest_rating,highest_rating))

Ratings range between 0.5 and 4.0


In [4]:
reader = surprise.Reader(rating_scale = (lowest_rating,highest_rating))
data = surprise.Dataset.load_from_df(ratings,reader)

In [5]:
similarity_options = {'name': 'cosine', 'user_based': True}

In [6]:
# Default k = 40
algo = surprise.KNNBasic(sim_options = similarity_options)
output = algo.fit(data.build_full_trainset())

Computing the cosine similarity matrix...
Done computing similarity matrix.


In [7]:
pred = algo.predict(uid='50',iid='6')
score = pred.est
print(score)

3.0028030537791928


In [9]:
iids = ratings['iid'].unique()

In [10]:
iids50 = ratings.loc[ratings['uid'] == 50 ,'iid']
print("List of iid that uid={0} has rated:".format(50))
print(iids50)

List of iid that uid=50 has rated:
864      8
865    211
866      3
867      2
868    219
869    234
870     12
871    254
872    250
873    207
874     11
875    253
876    236
877     84
878     10
879      7
880    233
881     13
882      1
883      5
884      6
885    252
886    241
887    216
888    257
889    206
890      4
891    217
892      9
893    215
894    213
895     17
896    255
897    220
898    121
899    245
900    239
901    251
902    235
Name: iid, dtype: int64


In [11]:
iids_to_predict = np.setdiff1d(iids,iids50)
print("List of iid which uid={0} did not rate(in all {1}) :".format(50,len(iids_to_predict)))
print(iids_to_predict)

List of iid which uid=50 did not rate(in all 2032) :
[  14   15   16 ... 2069 2070 2071]


In [12]:
### ratings arbitrarily set to 0
testset = [[50,iid,0.] for iid in iids_to_predict]
predictions = algo.test(testset)
predictions[5]

Prediction(uid=50, iid=20, r_ui=0.0, est=2.502007451844843, details={'actual_k': 7, 'was_impossible': False})

In [13]:
pred_ratings = np.array([pred.est for pred in predictions])

In [14]:
# Finding the index of maximum predicted rating
i_max = pred_ratings.argmax()

In [15]:
# Recommending the item with maximum predicted rating
iid_recommend_most = iids_to_predict[i_max] 
print("Top item to be recommended for user {0} is {1} with predicted rating as {2}".format(50,iid_recommend_most,pred_ratings[i_max]))

Top item to be recommended for user 50 is 35 with predicted rating as 4.0


In [16]:
# Getting top 10 items to be recommended for uid = 50
import heapq
i_sorted_10 = heapq.nlargest(10, 
                             range(len(pred_ratings)), 
                             pred_ratings.take)
top_10_items = iids_to_predict[i_sorted_10]
print(top_10_items)

[ 35  54  68  97 107 111 118 136 162 228]


In [17]:
############ Tuning ############

from surprise.model_selection import GridSearchCV
param_grid = {'k': np.arange(10,60,5)}

In [18]:
from surprise.model_selection.split import KFold
kfold = KFold(n_splits=5, random_state=2021, shuffle=True)
gs = GridSearchCV(surprise.KNNBasic, param_grid, 
                  measures=['rmse', 'mae'], cv=kfold)

In [ ]:
gs.fit(data)

Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.


In [ ]:
# best RMSE score
print(gs.best_score['rmse'])

In [ ]:
# combination of parameters that gave the best RMSE score
print(gs.best_params['rmse'])

In [ ]:
# We can now use the algorithm that yields the best rmse:
algo = gs.best_estimator['rmse']

In [ ]:
algo.fit(data.build_full_trainset())

In [ ]:
######################################

pred = algo.predict(uid='66',iid='207')
score = pred.est
print(score)

In [ ]:
iids = ratings['iid'].unique()
iids50 = ratings.loc[ratings['uid'] == 66 ,'iid']
print("List of iid that uid={0} has rated:".format(66))
print(iids50)

In [ ]:
iids_to_predict = np.setdiff1d(iids,iids50)
print("List of iid which uid={0} did not rate(in all {1}) :".format(66,len(iids_to_predict)))
print(iids_to_predict)

In [ ]:
### ratings arbitrarily set to 0
testset = [[66,iid,0.] for iid in iids_to_predict]
predictions = algo.test(testset)
predictions[5]

In [ ]:
pred_ratings = np.array([pred.est for pred in predictions])

In [ ]:
# Finding the index of maximum predicted rating
i_max = pred_ratings.argmax()

In [ ]:
# Recommending the item with maximum predicted rating
iid_recommend_most = iids_to_predict[i_max] 
print("Top item to be recommended for user {0} is {1} with predicted rating as {2}".format(50,iid_recommend_most,pred_ratings[i_max]))

In [ ]:
# Getting top 10 items to be recommended for uid = 50
import heapq
i_sorted_10 = heapq.nlargest(10, range(len(pred_ratings)), pred_ratings.take)
top_10_items = iids_to_predict[i_sorted_10]
print(top_10_items)